#### 1. **Model Runner Step Sync And Async Testing**

Notebook tests `ModelRunnerStep` within a serving graph to enable real-time monitoring and drift detection with MLRun.
The main focus is to test te preformance of sync and async usage.

In [9]:
# Import mlrun and create project instance
import mlrun

image = "mlrun/mlrun"
mlrun.set_env_from_file('../../cust_cs.env')
project_name = "monitored-model-runner-naive"
project = mlrun.get_or_create_project(project_name, context="./",user_project=True, allow_cross_project=True)

> 2025-10-29 12:06:54,964 [info] Server and client versions are not the same but compatible: {"parsed_client_version":"Version(major=1, minor=9, patch=2, prerelease=None, build=None)","parsed_server_version":"Version(major=1, minor=10, patch=0, prerelease='rc37', build=None)"}
> 2025-10-29 12:06:55,139 [info] Project loaded successfully: {"project_name":"monitored-model-runner-naive-matanz"}


In [4]:
# Import tools
import pandas as pd
from sklearn.svm import SVC
import pickle
from sklearn.datasets import load_iris
from mlrun.features import Feature

In [5]:
# Train simple SVM model on Iris dataset, save it and reformat the DS as list
iris = load_iris()
clf = SVC()           
clf.fit(iris.data, iris.target)
with open("../SVM.pkl", "wb") as fh:
    pickle.dump(clf, fh)
iris_data = iris["data"].tolist()

# load the dataset again as a DF
iris = load_iris()
train_set = pd.DataFrame(
    iris["data"],
    columns=["sepal_length_cm", "sepal_width_cm", "petal_length_cm", "petal_width_cm"],
)

# Create a Model Artifact in the project using the trained model
model_name = "SVM"
model_artifact = project.log_model(
    model_name,
    model_file="../SVM.pkl",
    training_set=train_set,
    framework="sklearn",
    outputs=[Feature(name="label")],
)

> 2025-10-28 17:31:34,509 [warning] Could not detect path to API server, not connected to API server!
> 2025-10-28 17:31:34,509 [warning] MLRUN_DBPATH is misconfigured. Set this environment variable to the URL of the API server in order to connect


#### 5. Define your function and ModelRunnerStep

Define functions to all the edge cases

In [6]:
# Config the code path and the serving function sync and async
sync_code_path = r"model_class_sync.py"
bouth_code_path = r"model_class_bouth.py"

async_sync_function = project.set_function(func=sync_code_path,image="mlrun/mlrun",kind="serving",name="async")
bouth_function = project.set_function(func=bouth_code_path,image="mlrun/mlrun",kind="serving",name="bouth")

In [7]:
from mlrun.serving.states import ModelRunnerStep

model_runner_step = ModelRunnerStep(
    name="my_runner", model_selector="MyModelSelector",model_selector_parameters={"name":"my-selector"})

for i in range(100):
    model_runner_step.add_model(
        model_class="MyModel",
        endpoint_name=f"my-{i}-model",
        model_artifact=model_artifact,
        input_path="inputs.here",
        result_path="outputs",
        outputs=["label"],
        execution_mechanism="asyncio",
    )

TypeError: __init__() got an unexpected keyword argument 'model_selector_parameters'

#### 6. Build graphs to all the edge cases

description


In [ ]:
async_graph_defined = bouth_function.set_topology("flow",engine="async")
async_graph_defined.to("MyPreprocessStep").to(model_runner_step).to("MyEnrichStep").respond()
async_graph_defined.plot()

async_graph_undefined = async_sync_function.set_topology("flow",engine="async")
async_graph_undefined.to("MyPreprocessStep").to(model_runner_step).to("MyEnrichStep").respond()
async_graph_undefined.plot()


#### Run using mock

descteption

In [ ]:
# 4. create mock server and test it locally (no k8s / deployment)

async_undefined_mock_server = bouth_function.to_mock_server()
async_defined_mock_server = async_sync_function.to_mock_server()

In [ ]:
async_undefined_mock_server
async_defined_mock_server


In [ ]:
from random import choice
from datetime import datetime

iris_data = iris["data"].tolist()
data_point = choice(iris_data)
print(f"Data point:{data_point}")


print("Before async_sync_function invoke:", datetime.now().strftime("%H:%M:%S.%f"))
async_sync_response = async_undefined_mock_server.test(
    "/",
    body={
        "models": None,
        "inputs": [data_point, data_point],
    },
)
print("After invoke:", datetime.now().strftime("%H:%M:%S.%f"))


In [ ]:
async_sync_response

In [ ]:
iris_data = iris["data"].tolist()
data_point = choice(iris_data)
print(f"Data point:{data_point}")


print("Before async_sync_function invoke:", datetime.now().strftime("%H:%M:%S.%f"))
async_sync_response = async_defined_mock_server.test(
    "/",
    body={
        "models": None,
        "inputs": [data_point, data_point],
    },
)
print("After invoke:", datetime.now().strftime("%H:%M:%S.%f"))


In [ ]:
async_sync_response

#### 7. Deploying Your Function

Running this cell will deploy your serving function to the cluster. This also deploys the real-time monitoring functions for your project, which are configured to track the serving function's performance and detect model drift.

In [ ]:
async_sync_function.deploy()
bouth_function.deploy()

In [ ]:
iris_data = iris["data"].tolist()
data_point = choice(iris_data)
print(f"Data point:{data_point}")


print("Before async_sync_function invoke:", datetime.now().strftime("%H:%M:%S.%f"))
async_sync_response = async_sync_function.invoke(
    "/",
    body={
        "models": None,
        "inputs": [data_point, data_point],
    },
)
print("After invoke:", datetime.now().strftime("%H:%M:%S.%f"))

In [ ]:
async_sync_response

In [ ]:
iris_data = iris["data"].tolist()
data_point = choice(iris_data)
print(f"Data point:{data_point}")


print("Before bouth_function invoke:", datetime.now().strftime("%H:%M:%S.%f"))
bouth_response = bouth_function.invoke(
    "/",
    body={
        "models": None,
        "inputs": [data_point, data_point],
    },
)
print("After invoke:", datetime.now().strftime("%H:%M:%S.%f"))

In [ ]:
bouth_response

In [ ]:
from random import choice

iris_data = iris["data"].tolist()
data_point = choice(iris_data)
print(f"Data point:{data_point}")
response = function.invoke(
    "/", body={"models": ['my-13-model','my-16-model'], "inputs": [data_point, data_point]}
)

In [ ]:
response

# TODO
2. Check if we get a warning when we try to run async when there is no "async predict" 
    
3. Check if the async work as expected

4. Try to add different prompts to the same model using async, and inspect the behavior
